<a href="https://colab.research.google.com/github/duynht/private-ai/blob/master/Section%202%20-%20Federated%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
!pip install syft

     |████████████████████████████████| 440kB 4.5MB/s 
     |████████████████████████████████| 133kB 14.8MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 9.0MB 11.4MB/s 
     |████████████████████████████████| 2.0MB 29.9MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 2.2MB 34.4MB/s 
     |████████████████████████████████| 450kB 29.3MB/s 
     |████████████████████████████████| 61kB 3.7MB/s 
     |████████████████████████████████| 4.0MB 33.1MB/s 
     |████████████████████████████████| 204kB 38.0MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 753.4MB 18kB/s 
     |████████████████████████████████| 1.8MB 34.4MB/s 
     |████████████████████████████████| 491kB 42.3MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 36.9MB 87kB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |

In [2]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [4]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

In [7]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [9]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
x = x.send(bob)

In [13]:
bob._objects

{71616846850: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

71616846850

In [16]:
x.id

14814616968

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:14814616968 -> bob:71616846850]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [22]:
# try this project here!

In [23]:
alice = sy.VirtualWorker(hook, id='alice')

In [24]:
x = th.tensor([1, 2, 3, 4])

In [25]:
x_ptr = x.send(bob, alice)

In [26]:
x_ptr.get()

[tensor([1, 2, 3, 4]), tensor([1, 2, 3, 4])]

In [27]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [28]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [30]:
x

(Wrapper)>[PointerTensor | me:81002947696 -> bob:40439202944]

In [31]:
y

(Wrapper)>[PointerTensor | me:83980379089 -> bob:61917854642]

In [32]:
z = x + y

In [33]:
z

(Wrapper)>[PointerTensor | me:99133896931 -> bob:32743137232]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:48930651902 -> bob:12036224423]

In [36]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [37]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [38]:
z = (x + y).sum()

In [39]:
z.backward()

(Wrapper)>[PointerTensor | me:36210301714 -> bob:95767712395]

In [40]:
x = x.get()

In [41]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [42]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [43]:
# try this project here!

In [44]:
input = th.tensor([[1., 1], [0, 1, ], [1, 0], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [45]:
input

(Wrapper)>[PointerTensor | me:89293272696 -> bob:21766642143]

In [46]:
target

(Wrapper)>[PointerTensor | me:55458916446 -> bob:83025244380]

In [47]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [48]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [49]:
bob = bob.clear_objects()

In [50]:
bob._objects

{}

In [51]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [52]:
bob._objects

{24770867742: tensor([1, 2, 3, 4, 5])}

In [53]:
del x

In [54]:
bob._objects

{}

In [55]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [56]:
x.child.garbage_collect_data

True

In [57]:
bob._objects

{77656156080: tensor([1, 2, 3, 4, 5])}

In [58]:
x = "asdf"

In [59]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
x

(Wrapper)>[PointerTensor | me:31434902352 -> bob:16901335663]

In [62]:
bob._objects

{16901335663: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
bob._objects

{16901335663: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
bob._objects

{16901335663: tensor([1, 2, 3, 4, 5])}

In [67]:
bob = bob.clear_objects()
bob._objects

{}

In [68]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [69]:
bob._objects

{85929751445: tensor([1, 2, 3, 4, 5])}

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [71]:
z = x + y

TensorsNotCollocatedException: ignored

In [72]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [73]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [74]:
from torch import nn, optim

In [75]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [76]:
# A Toy Model
model = nn.Linear(2,1)

In [77]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [78]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.7863)
tensor(1.2270)
tensor(0.8554)
tensor(0.6015)
tensor(0.4263)
tensor(0.3045)
tensor(0.2190)
tensor(0.1587)
tensor(0.1157)
tensor(0.0849)
tensor(0.0626)
tensor(0.0464)
tensor(0.0345)
tensor(0.0258)
tensor(0.0193)
tensor(0.0145)
tensor(0.0110)
tensor(0.0083)
tensor(0.0063)
tensor(0.0047)


In [79]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [80]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [81]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [82]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [83]:
train()

tensor(1.1670, requires_grad=True)
tensor(1.7890, requires_grad=True)
tensor(0.5971, requires_grad=True)
tensor(1.1559, requires_grad=True)
tensor(0.3764, requires_grad=True)
tensor(0.7489, requires_grad=True)
tensor(0.2452, requires_grad=True)
tensor(0.4910, requires_grad=True)
tensor(0.1630, requires_grad=True)
tensor(0.3260, requires_grad=True)
tensor(0.1105, requires_grad=True)
tensor(0.2192, requires_grad=True)
tensor(0.0762, requires_grad=True)
tensor(0.1491, requires_grad=True)
tensor(0.0535, requires_grad=True)
tensor(0.1026, requires_grad=True)
tensor(0.0380, requires_grad=True)
tensor(0.0714, requires_grad=True)
tensor(0.0274, requires_grad=True)
tensor(0.0501, requires_grad=True)
tensor(0.0199, requires_grad=True)
tensor(0.0355, requires_grad=True)
tensor(0.0146, requires_grad=True)
tensor(0.0254, requires_grad=True)
tensor(0.0108, requires_grad=True)
tensor(0.0182, requires_grad=True)
tensor(0.0080, requires_grad=True)
tensor(0.0132, requires_grad=True)
tensor(0.0059, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [84]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [86]:
x = x.send(alice)

In [87]:
bob._objects

{89116414664: tensor([1, 2, 3, 4, 5])}

In [88]:
alice._objects

{7633418841: (Wrapper)>[PointerTensor | alice:7633418841 -> bob:89116414664]}

In [89]:
y = x + x

In [90]:
y

(Wrapper)>[PointerTensor | me:89170870472 -> alice:32413295045]

In [91]:
bob._objects

{23125395796: tensor([ 2,  4,  6,  8, 10]),
 89116414664: tensor([1, 2, 3, 4, 5])}

In [92]:
alice._objects

{7633418841: (Wrapper)>[PointerTensor | alice:7633418841 -> bob:89116414664],
 32413295045: (Wrapper)>[PointerTensor | alice:32413295045 -> bob:23125395796]}

In [93]:
jon = sy.VirtualWorker(hook, id="jon")

In [94]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [95]:
bob._objects

{51357942563: tensor([1, 2, 3, 4, 5])}

In [96]:
alice._objects

{97136574053: (Wrapper)>[PointerTensor | alice:97136574053 -> bob:51357942563]}

In [97]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:97136574053 -> bob:51357942563]

In [98]:
bob._objects

{51357942563: tensor([1, 2, 3, 4, 5])}

In [99]:
alice._objects

{}

In [100]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [101]:
bob._objects

{}

In [102]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [103]:
bob._objects

{10879606817: tensor([1, 2, 3, 4, 5])}

In [104]:
alice._objects

{94450960286: (Wrapper)>[PointerTensor | alice:94450960286 -> bob:10879606817]}

In [105]:
del x

In [106]:
bob._objects

{}

In [107]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [108]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [109]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [110]:
bob._objects

{66868868486: tensor([1, 2, 3, 4, 5])}

In [111]:
alice._objects

{}

In [112]:
x.move(alice)

(Wrapper)>[PointerTensor | me:48948943239 -> alice:66868868486]

In [113]:
bob._objects

{}

In [114]:
alice._objects

{66868868486: tensor([1, 2, 3, 4, 5])}

In [115]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [116]:
bob._objects

{8353205035: tensor([1, 2, 3, 4, 5])}

In [117]:
alice._objects

{11922500274: (Wrapper)>[PointerTensor | alice:11922500274 -> bob:8353205035],
 66868868486: tensor([1, 2, 3, 4, 5])}

In [118]:
x.remote_get()

(Wrapper)>[PointerTensor | me:48655488781 -> alice:11922500274]

In [119]:
bob._objects

{}

In [120]:
alice._objects

{11922500274: tensor([1, 2, 3, 4, 5]), 66868868486: tensor([1, 2, 3, 4, 5])}

In [121]:
x.move(bob)

(Wrapper)>[PointerTensor | me:48655488781 -> bob:11922500274]

In [122]:
x

(Wrapper)>[PointerTensor | me:48655488781 -> bob:11922500274]

In [123]:
bob._objects

{11922500274: tensor([1, 2, 3, 4, 5])}

In [124]:
alice._objects

{66868868486: tensor([1, 2, 3, 4, 5])}

# Section Project:

For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form

In [1]:
!pip install syft

     |████████████████████████████████| 440kB 4.7MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 9.0MB 57.6MB/s 
     |████████████████████████████████| 450kB 48.5MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 133kB 52.5MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 4.0MB 44.3MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 204kB 61.9MB/s 
     |████████████████████████████████| 491kB 49.0MB/s 
     |████████████████████████████████| 2.0MB 47.7MB/s 
     |████████████████████████████████| 1.8MB 46.6MB/s 
     |████████████████████████████████| 2.2MB 44.8MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     

In [41]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
std_transforms = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,)),
                                    ])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=std_transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=std_transforms)

In [42]:
import syft
import torch

In [43]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

In [44]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [49]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    total_loss = 0
    total_acc = 0
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back

        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [50]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [53]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 300
        self.save_model = False

args = Arguments()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
worker_list = [syft.VirtualWorker(syft.TorchHook(torch), id=str(id)) for id in range(10)]
federated_train_loader = syft.FederatedDataLoader(mnist_trainset.federate(worker_list), batch_size=args.batch_size, shuffle=True)
test_loader = data.DataLoader(mnist_testset, batch_size=args.test_batch_size, shuffle=False, drop_last=True)

In [55]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.308829
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.594028
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.203868
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.210726

Test set: Average loss: 0.1903, Accuracy: 9426/10000 (94%)

Train Epoch: 2 [0/60032 (0%)]	Loss: 0.375240
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.122242
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.086708
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.084191

Test set: Average loss: 0.1121, Accuracy: 9656/10000 (97%)

Train Epoch: 3 [0/60032 (0%)]	Loss: 0.128822
Train Epoch: 3 [19200/60032 (32%)]	Loss: 0.066789
Train Epoch: 3 [38400/60032 (64%)]	Loss: 0.070911
Train Epoch: 3 [57600/60032 (96%)]	Loss: 0.020738

Test set: Average loss: 0.0849, Accuracy: 9735/10000 (97%)

Train Epoch: 4 [0/60032 (0%)]	Loss: 0.049208
Train Epoch: 4 [19200/60032 (32%)]	Loss: 0.053748
Train Epoch: 4 [38400/60032 (64%)]	Loss: 0.059955
Train Epoch: 4 [57600/60032 (96%)]	Loss: 0.039767

Test set: Average loss: 0.0625, Accu